# library & data loading

In [1]:
import pandas as pd
import numpy as np
import re

In [ ]:
# data loading file
df = pd.read_excel('') # 파일경로
df['txtb_id'] = df['isbn'].apply(lambda x : f'm{x}') # txtb_id 
df['set_isbn'] = df['set_isbn'].replace(np.NaN,0).astype(str).apply(lambda x: x[:-2]) # 널값 대체 및 타입변경

# 타깃 분리 및 단권, 세트권 분리
total = df.loc[df['id'].isin(target)] # target 존재시
onePart = total.loc[total['set_isbn'] == '0', :] # 단권
setPart = total.loc[total['set_isbn']!='0',:] # 세트권

# 인덱스 초기화
onePart.index = range(len(onePart)) 
setPart.index = range(len(setPart))

# function defining

In [ ]:
## A
def index_classification(isbn, tit, text, i, ex):
    code = isbn[i] # isbn, main_title
#     title = tit[i]
#     d1,d2,d3,d4 = 0,0,0,0  # 뎁스 채울 변수(초기화)
    if '목차 더보기' in text: # 잘못 들어간 단어 삭제(틀 전체로 접근하다 보니 같이 달려옴)
        text.remove('목차 더보기')
    text = [clear_text(r) for r in text] # 2
    text = [delete_text(r) for r in text] # 3
    splitKwd,index_ls, splitKwd_ls, page_ls,etc_ls = classification_contents(text,ex)
    if '총론' in splitKwd and '각론' in splitKwd:
        splitKwd.remove('각론')
        splitKwd_ls = ['총론' if c == '각론' else c for c in splitKwd_ls]
    
    if 'check' not in splitKwd:
        tcntnID = [f'n{f"{i}".zfill(3)}' for i in range(len(splitKwd_ls))] # 바로저장으로 수정+ 전달도 있어야 함
        dep = assign_depth(splitKwd_ls)
        bin_ls = circle(splitKwd_ls,splitKwd)
        high = assign_highID(tcntnID, dep)#assign_floor_index(myID, dep) # assign_high(ck_ls, myID, pandan) # high_depthID(ck_ls, pandan,myID)
        study = assign_study(dep) 
        
        no_ls = notStudy_change(index_ls)
        no_lss = zero_ls(bin_ls, no_ls)
        check_study(study, dep, etc_ls)
        print(no_lss)
        try:
            zero_change(high, dep, study, no_lss)
        except:
            pass
        
        for ti in tcntnID:
            mokchaID.append(ti)
        for ii in index_ls:
            contents.append(ii)
        for pi in page_ls:
            start_pn.append(pi)
        for di in dep:
            depth.append(di)
        for hi in high:
            high_mokchaID.append(hi)
        for si in study:
            learning.append(si)   
        for ei in etc_ls:
            etc.append(ei)
        for j in range(len(index_ls)):
            bookID.append(code)
    else:
        noList.append(code)  # 수정함
#         etc.append('check')
#         for j in [mokchaID, high_mokchaID, contents, learning, depth, start_pn, etc]:
#             j.append(np.Na

In [ ]:
## 정제 함수
# 1
def make_ls(c,i): # 인덱스 리스트와 순서를 받음
    matches = c[i].split('-+-')
    return matches
# # 1번의 이전 버전
# def make_ls(c,i): # 인덱스 리스트와 순서를 받음
#     matches = re.findall(r"'(.*?)'", c[i])
#     return list(matches)

# 2
def clear_text(sentence):  # 목차 내용 정제 함수
    pattern = r'[\w가-힣]+'
    extracted = re.findall(pattern, sentence)
    result = ' '.join(extracted)
    return result

# 3
def delete_text(sentence):
    pattern = r'u3000|u000|u00|uf0ab|u2005|'#목차 더보기'
    result = re.sub(pattern, '', sentence)
    return result

In [ ]:
# 판별 함수
# 1
def classification_contents(text, ex): # 판별 요소 설정 함수
    pd_l,ck_l, ck_ls,pn_l, etc_l = [],[],[],[],[]
    for idx, line in enumerate(text):
        a = re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S', '', line).strip()
        # 숫자 판별
        l, page, k= split_page(line)
        if line[0].isnumeric() == True: 
            if a != '' and 'roma' not in pd_l: # 로마 숫자
                ck_l.append(l) # 위치 저장
                pn_l.append(page)
                ck_ls.append('roma')
                pd_l.append('roma')
                o = classification_roma(line)
                etc_l.append([o,k])
            elif a != '' and 'roma' in pd_l: # 로마 숫자
                ck_l.append(l) # 위치 저장
                pn_l.append(page)
                ck_ls.append('roma')
                o = classification_roma(line)
                etc_l.append([o,k])
            elif a == '' and line[1] in ['부', '절','편','강'] and 'num' not in pd_l :
                ck_l.append(l)
                pn_l.append(page)
                ck_ls.append('num')
                pd_l.append('num') # 분류랑 숫자 결합
                o = classification_num(line)
                etc_l.append([o,k])
            elif a == '' and line[1] in ['부', '절','편','강'] and 'num' in pd_l:
                ck_l.append(l)
                pn_l.append(page)
                ck_ls.append('num')
                o = classification_num(line)
                etc_l.append([o,k])
            elif a == '' and line[1] not in ['부', '절','편','강'] and 'ara' not in pd_l :
                ck_l.append(l)
                pn_l.append(page)
                ck_ls.append('ara')
                pd_l.append('ara') # 분류랑 숫자 결합
                o = classification_ara(line)
                etc_l.append([o,k])
            elif a == '' and line[1] not in ['부', '절','편','강'] and 'ara' in pd_l:
                ck_l.append(l)
                pn_l.append(page)
                ck_ls.append('ara')
                o = classification_ara(line)
                etc_l.append([o,k])
        elif line[0].isnumeric() != True:   # 포함되는 리스트
            t = re.sub(r'[^a-zA-Z가-힣]','',line.lower())  # 기록 시에는 line을 기록해야 함
            b = ex.match(t)
            if b and b.group() not in pd_l:
                pd_l.append(b.group())
                ck_l.append(l)
                pn_l.append(page)
                ck_ls.append(b.group())
                o = classification_ex(line, b)
                etc_l.append([o,k])
            elif b and b.group() in pd_l:
                ck_l.append(l)
                pn_l.append(page)
                ck_ls.append(b.group())
                o = classification_ex(line, b)
                etc_l.append([o,k])
                pass
            else:
                pd_l.append('check')
                
    return pd_l, ck_l,ck_ls, pn_l,etc_l

# 2
def classification_roma(line):
    o =''
    test = line.split(' ')
    test = [1 for t in test if re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S','', t) != '']
    if sum(test)> 1:
        o = 'over'
    else:
        o = 0
    return o

# 3
def classification_ara(line):
    o =''
    test = line.split(' ')
    test = [1 for t in test if t.isnumeric() == True]
    if sum(test)> 1:
        o = 'over'
    else:
        o = 0
    return o    

# 4
def classification_num(line):
    o =''
    if line.count(line[1]) > 1:
        o = 'over'
    else:
        o = 0
    return o    

# 5
def classification_ex(line, b):
    o =''
    if line.count(b.group())>1:
        o = 'over'
    else:
        o = 0  
    return o

# 6
def split_page(line): # 페이지 분리 함수 |Ⅸ
    texts = line.split(' ')
    if len(texts) >= 2:
        l = texts[-1]
        ll = texts[-2]
        a = re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S', '', l).strip()
        b = re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S', '', ll).strip()
        if l.isnumeric() == True and a == '':
            if ll.isnumeric() == True and b == '':
                page = l
                line = ' '.join(line.split(' ')[:-1])
                if len(page) > 3:
                    k = 'pn'
                else:
                    k = 'pn'
            elif ll.isnumeric() == True and b != '':
                page = l
                line = ' '.join(line.split(' ')[:-1])
                if len(page) > 3:
                    k = 'pn'
                else:
                    k = 0
            else:
                page = l
                line = ' '.join(line.split(' ')[:-1])
                if len(page) > 3:
                    k = 'pn'
                else:
                    k = 0
        else:
            page = np.NaN
            k = 0
    else:
        l = texts[-1]
        a = re.sub(r'(?![ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ])\S', '', l).strip()
        if l.isnumeric() == True and a == '':
            page = l
            line = ' '.join(line.split(' ')[:-1])
            if len(page) > 3:
                k = 'pn'
            else:
                k = 0
        else:
            page = np.NaN
            k = 0
    return line, page, k 


In [ ]:
# 작업 함수

# 1
def circle(ck_ls,splitKwd):  # 구역 판별
    bin_ls = []
    x = -1
    for i in ck_ls:
        if i == splitKwd[0]:
            x += 1
            bin_ls.append(x)
        else:
            bin_ls.append(x)
    return bin_l

# 2
def assign_study(depth):
    study = []
    for i in range(len(depth)):
        if i == 0:
            if depth[i] == depth[i+1]:
                study.append('Y')
            else:
                study.append('N')
        elif i != len(depth)-1:
            if depth[i] == depth[i+1] or depth[i] == depth[i-1]:
                study.append('Y')
            else:
                study.append('N')
        else:
            study.append(study[-1])
    return study

# 3
def assign_depth(lst):  # 뎁스 판별
    dep_ls = []
    dep = 1
    depth_map = {}
    for element in lst:
        if element not in depth_map:
            depth_map[element] = dep
            dep += 1
        dep_ls.append(depth_map[element])
    return dep_ls

# 4
def assign_highID(myID, dep): # 상위 뎁스 판단 함수
    high = []
    depth_dic = {}
    for i in range(len(dep)):
        if dep[i] == 1:
            high.append('')
            depth_dic[dep[i]] = myID[i]
        elif dep[i] == 2:
            high.append(depth_dic[1])
            depth_dic[dep[i]] = myID[i]
        elif dep[i] == 3:
            high.append(depth_dic[2])
            depth_dic[dep[i]] = myID[i]
        elif dep[i] == 4:
            high.append(depth_dic[3])
            depth_dic[dep[i]] = myID[i]
        elif dep[i] == 5:
            high.append(depth_dic[4])
            depth_dic[dep[i]] = myID[i]
        else:
            high.append('?')
    return high

In [ ]:
## 후속 함수      
# 1 
def notStudy_change(contents): # 학습연계가 N인 요소 판별(분류된것중 해당 단어 체크)
    ls = []
    a = re.compile('부록')
    b = re.compile('정답|해설|색인')
    for i,v in enumerate(contents):
        if a.match(v):
            ls.append(i)
        elif b.search(v):
            ls.append(i)
        else:
            pass
        
    return ls

# 2
def zero_ls(circle, ls):
    t = ''
    if len(ls) == 1:
        t = [i for i in range(ls[0],len(circle)+1)]
    elif len(ls) > 1:
        t = []
        for l in ls:
            ck = circle[l]
            zero_plus(circle, l, ck,t)
            
    else:
        t = 'X' # 바꿀 정보 없음
    return t

# 3
def zero_change(high, dep, study, t): #
    if t == 'X':
        pass
    elif len(t)>1:
        for ti in t:
            high[ti] = ''
            dep[ti] = ''
            study[ti] = 'N'

# 4
def zero_plus(circle, l, ck, t):   ## zero_ls의 후속함수
    for i, v in enumerate(circle):
        if i >= l and v == ck:
            t.append(i)
        else:
            pass 

# 5
def check_study(study, dep, etc_l):
    for i in len(study):
        if i != 0 and i != len(study):
            if set(study[i-1:i+2]) == {'N'} and len(set(dep[i-1:i+2])) == 2:
                for j in range(i-1, i+2):
                    etc_l[j].append('std_lnk')

# action -  단권 자동화

In [ ]:
# 변수 설정
isbn, tit, text = onePart['txtb_id'], onePart['set_isbn'], onePart['index']
# 저장될 리스트
bookID, mokchaID, high_mokchaID, contents, learning, depth,etc= [],[],[],[],[],[],[]
start_pn, end_pn = [],[]
noList = []

# 목차 구분자
ex = re.compile("제절|제편|제강|제항|제장|제회|제부|제과|제관|강|목|장|항|회|부|사례|총론|각론|부록|키워드|최근|실전|모의|문제|정답|해설|차례|전범위|참고|판례|핵심|시행|약점|연습|기출|t|book|part|chapter|day|section|theme|case|keyword|point|q|question|contents")

In [ ]:
# 실행
for i in range(len(text)):
    try:
        tet = make_ls(text, i)
        index_classification(isbn, tit, tet, i, ex)
    except:
#         print('i')
        noList.append(isbn[i])

In [ ]:
data = pd.DataFrame({'txtb_id':bookID,'tcntn_id':mokchaID, 'uppr_tcntn_id':high_mokchaID,'tcntn_trn':etc,\
                   'tcntn_nm':contents, 'std_lnk_yn':learning, 'depth':depth,'str_page':start_pn})
# end_page 열 만드는 작업
end = start_pn[1:]
end.append(np.NaN)
data['end_page'] = end # 해당컬럼 추가
data.loc[data['str_page'].isna(),'end_page'] = np.NaN # 데이터 정제

data

In [ ]:
## 정제 데이터 저장 
# data.to_excel("C:/Users/sub computer/Desktop/추가_단권_목차.xlsx",encoding = 'utf-8-sig', index=False)

In [ ]:
# 자동화되지 않은 데이터 구분
noOne = onePart.loc[onePart['txtb_id'].isin(noList),:]
noOne.index = range(len(noOne))
noOne.info()

In [ ]:
# 자동화 분류 개수
len(onePart) - len(noOne) 

# action - 단권 수작업

In [ ]:
noOne # 타깃 대상

In [ ]:
# 목차 분류 구분자
tex = re.compile("in|book|찾|주제|제절|제편|제강|제항|제장|제회|제부|제과|제관|강|목|장|항|회|부|사례|총론|각론|부록|키워드|최근|실전|모의|문제|정답|해설|차례|전범위|참고|판례|핵심|시행|약점|연습|기출|t|book|part|chapter|day|section|theme|case|keyword|point|q|question|contents")

In [ ]:
# 초기화
isbn, tit, text = noOne['isbn'], noOne['set_isbn'], noOne['index']

bookID, mokchaID, high_mokchaID, contents, learning, depth,etc= [],[],[],[],[],[],[]
start_pn, end_pn = [],[]

In [ ]:
num =  # 대상 인덱스
code = isbn[num]
text = make_ls(text, num)
print(code)
text
# 해당 데이터 형태를 확인하여 반자동화(1) 또는 올수작업(2)을 선택하여 작업

### 반자동화
- 작업을 위한 구분자 추가
- 분류되지 않아도 무방한 대상이 존재시 체크 부분을 통해 제어

In [ ]:
if '목차 더보기' in text: # 잘못 들어간 단어 삭제(틀 전체로 접근하다 보니 같이 달려옴)
    text.remove('목차 더보기')
    print(0)
text = [clear_text(r) for r in text] # 이상한 문자 제거
text = [delete_text(r) for r in text]
print(1)
splitKwd,index_ls, splitKwd_ls, page_ls,etc_ls = classification_contents(text,tex) # 구분자 체크
print(2,splitKwd)
if '총론' in splitKwd and '각론' in splitKwd:
    splitKwd.remove('각론')
    splitKwd_ls = ['총론' if c == '각론' else c for c in splitKwd_ls]
# 체크    
if 'check' not in splitKwd: # if 'check' in pandan:
    tcntnID = [f'n{f"{i}".zfill(4)}' for i in range(len(splitKwd_ls))] 
    dep = assign_depth(splitKwd_ls)
    print(3)
    bin_ls = circle(splitKwd_ls,splitKwd)
    high = assign_highID(tcntnID, dep)
    study = assign_study(dep) 
    print(4)
    no_ls = notStudy_change(index_ls)
    no_lss = zero_ls(bin_ls, no_ls)
    print(no_lss)
    try:
        zero_change(high, dep, study, no_lss)
    except:
        pass
        
    for ti in tcntnID:
        mokchaID.append(ti)
    for ii in index_ls:
        contents.append(ii)
    for pi in page_ls:
        start_pn.append(pi)
    for di in dep:
        depth.append(di)
    for hi in high:
        high_mokchaID.append(hi)
    for si in study:
        learning.append(si)   
    for ei in etc_ls:
        etc.append(ei)
    for j in range(len(index_ls)):
        bookID.append(f'm{code}')

In [ ]:
test = pd.DataFrame({'txtb_id':bookID,'tcntn_id':mokchaID, 'uppr_tcntn_id':high_mokchaID,'tcntn_trn':etc, \
                   'tcntn_nm':contents,'std_ln_yn':learning,'depth':depth, 'str_page':start_pn})
end = start_pn[1:]
end.append(np.NaN)
test # 결과 확인

### 올수작업
- 텍스트와 페이지 넘버 분리만 수행

In [ ]:
line, page = [],[]
if '목차 더보기' in text: # 잘못 들어간 단어 삭제(틀 전체로 접근하다 보니 같이 달려옴)
    text.remove('목차 더보기')
    print(0)
text = [clear_text(r) for r in text] # 이상한 문자 제거
text = [delete_text(r) for r in text]
for t in text:
    l, pn, k= split_pn2(t)
    line.append(l)
    page.append(pn)
    
test = pd.DataFrame({'line':line,'page':page})
test['code'] = f'm{code}'
test

### 저장 

In [ ]:
# test.to_excel("C:/Users/sub computer/Desktop/추가단권_수작업++.xlsx",encoding = 'utf-8-sig', index=False)

# action - 세트 자동화

In [ ]:
# 세트권 유니크값 리스트화
setp = setPart['set_isbn'].unique().tolist()

# 세트별 개별권 데이터 프레임 
setp_ls = []
for sp in setp:
    x = setPart.loc[setPart['set_isbn'] == sp, 'isbn'].tolist()
    setp_ls.append(x)
    
setp_df = pd.DataFrame({'set_isbn': setp, 'isbn_ls' : setp_ls})
setp_df

In [ ]:
# 타깃 정리
setp_df = pd.merge(setp_df, setPart[['isbn','txtb_nm','index']], left_on= 'set_isbn',\
                   right_on = 'isbn')


In [ ]:
# 변수 설정
isbn, text = setp_df['isbn'],  setp_df['index']
# 저장될 리스트
bookID, mokchaID, high_mokchaID, contents, learning, depth,etc= [],[],[],[],[],[],[]
start_pn, end_pn = [],[]
noList = []

ex = re.compile("제절|제편|제강|제항|제장|제회|제부|제과|제관|강|목|장|항|회|부|사례|총론|각론|부록|키워드|최근|실전|모의|문제|정답|해설|차례|전범위|참고|판례|핵심|시행|약점|연습|기출|t|book|part|chapter|day|section|theme|case|keyword|point|q|question|contents")

In [ ]:
# 실행
for i in range(len(text)):
    try:
        tet = make_ls(text, i)
        clear_idx(isbn, tit, tet, i, ex)
    except:
#         print('i')
        noList.append(isbn[i])
    
data = pd.DataFrame({'txtb_id':bookID,'tcntn_id':mokchaID, 'uppr_tcntn_id':high_mokchaID,'tcntn_trn':etc,\
                   'tcntn_nm':contents, 'stdy_lnk_yn':learning, 'depth':depth,'str_page':start_pn})
data

In [ ]:
noSet = setp_df.loc[setp_df['isbn'].isin(noList),:]
noSet.index = range(len(noSet))
noSet.info()

In [ ]:
len(setp_df) - len(noSet)

In [ ]:
## 저장
# data.to_excel("C:/Users/sub computer/Desktop/경찰나머지_세트_목차.xlsx",encoding = 'utf-8-sig', index=False)

# action - 세트 수작업

In [ ]:
# 목차 분류 구분자
tex = re.compile("in|book|찾|주제|제절|제편|제강|제항|제장|제회|제부|제과|제관|강|목|장|항|회|부|사례|총론|각론|부록|키워드|최근|실전|모의|문제|정답|해설|차례|전범위|참고|판례|핵심|시행|약점|연습|기출|t|book|part|chapter|day|section|theme|case|keyword|point|q|question|contents")

In [ ]:
# 초기화
isbn, tit, text = noSet['isbn'], noSet['set_isbn'], noSet['index']

bookID, mokchaID, high_mokchaID, contents, learning, depth,etc= [],[],[],[],[],[],[]
start_pn, end_pn = [],[]

In [ ]:
num =  # 대상 인덱스
code = isbn[num]
text = make_ls(text, num)
print(code)
text
# 해당 데이터 형태를 확인하여 반자동화(1) 또는 올수작업(2)을 선택하여 작업

### 반자동화

In [ ]:
if '목차 더보기' in text: # 잘못 들어간 단어 삭제(틀 전체로 접근하다 보니 같이 달려옴)
    text.remove('목차 더보기')
    print(0)
text = [clear_text(r) for r in text] # 이상한 문자 제거
text = [delete_text(r) for r in text]
print(1)
splitKwd,index_ls, splitKwd_ls, page_ls,etc_ls = classification_contents(text,tex) # 구분자 체크
print(2,splitKwd)
if '총론' in splitKwd and '각론' in splitKwd:
    splitKwd.remove('각론')
    splitKwd_ls = ['총론' if c == '각론' else c for c in splitKwd_ls]
# 체크    
if 'check' not in splitKwd: # if 'check' in pandan:
    tcntnID = [f'n{f"{i}".zfill(3)}' for i in range(len(splitKwd_ls))] 
    dep = assign_depth(splitKwd_ls)
    print(3)
    bin_ls = circle(splitKwd_ls,splitKwd)
    high = assign_highID(tcntnID, dep)
    study = assign_study(dep) 
    print(4)
    no_ls = notStudy_change(index_ls)
    no_lss = zero_ls(bin_ls, no_ls)
    print(no_lss)
    try:
        zero_change(high, dep, study, no_lss)
    except:
        pass
        
    for ti in tcntnID:
        mokchaID.append(ti)
    for ii in index_ls:
        contents.append(ii)
    for pi in page_ls:
        start_pn.append(pi)
    for di in dep:
        depth.append(di)
    for hi in high:
        high_mokchaID.append(hi)
    for si in study:
        learning.append(si)   
    for ei in etc_ls:
        etc.append(ei)
    for j in range(len(index_ls)):
        bookID.append(f'm{code}')

In [ ]:
test = pd.DataFrame({'txtb_id':bookID,'tcntn_id':mokchaID, 'uppr_tcntn_id':high_mokchaID,'tcntn_trn':etc, \
                   'tcntn_nm':contents,'std_ln_yn':learning,'depth':depth, 'str_page':start_pn})
end = start_pn[1:]
end.append(np.NaN)
test['end_page'] = end # 해당컬럼 추가
test.loc[data['str_page'].isna(),'end_page'] = np.NaN # 데이터 정제

test # 결과 확인

### 올수작업

In [ ]:
line, page = [],[]
if '목차 더보기' in text: # 잘못 들어간 단어 삭제(틀 전체로 접근하다 보니 같이 달려옴)
    text.remove('목차 더보기')
    print(0)
text = [clear_text(r) for r in text] # 이상한 문자 제거
text = [delete_text(r) for r in text]
for t in text:
    l, pn, k= split_page(t)
    line.append(l)
    page.append(pn)
    
test = pd.DataFrame({'line':line,'page':page})
test['code'] = f'm{code}'
test

### 저장

In [2]:
# test.to_excel("C:/Users/sub computer/Desktop/경찰나머지_세트_수작업+.xlsx",encoding = 'utf-8-sig', index=False)